In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import os

In [4]:
work_dir = '/content/drive/MyDrive/AT SCHOOL/HK7/CS114/Final Project/'
train_dir = os.path.join(work_dir, 'Data/Train_test_val/Train/')
test_dir = os.path.join(work_dir, 'Data/Train_test_val/Test/')
val_dir = os.path.join(work_dir, 'Data/Train_test_val/Val/')

easyocr_dir = os.path.join(work_dir, 'Prediction/EASYOCR/image/')
easyocr_non_bgr_dir = os.path.join(work_dir, 'Prediction/EASYOCR/non_background/')
easyocr_non_bgr_pre_dir = os.path.join(work_dir, 'Prediction/EASYOCR/points_text_non_background/')
easyocr_pre_dir = os.path.join(work_dir, 'Prediction/EASYOCR/points_text/')
model_dir = os.path.join(work_dir, 'Model/')

##install

In [5]:
import ast
import pandas as pd
from tqdm import tqdm
import time
import numpy as np
import os
from shapely.geometry import Polygon
from statistics import mean
from os import system

In [6]:
!pip install jiwer
from jiwer import wer, cer

     |████████████████████████████████| 50 kB 2.8 MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149860 sha256=14908881cfd74bde3ad90ce4c48cf7a83b4cdd45b026ac61da8d84b3326cbb7e
  Stored in directory: /root/.cache/pip/wheels/05/5f/ca/7c4367734892581bb5ff896f15027a932c551080b2abd3e00d
Successfully built python-Levenshtein


In [7]:
!pip install easyocr

     |████████████████████████████████| 63.6 MB 61 kB/s 
     |████████████████████████████████| 47.7 MB 3.8 MB/s 


In [8]:
!pip install git+git://github.com/jaidedai/easyocr.git

  Cloning git://github.com/jaidedai/easyocr.git to /tmp/pip-req-build-nqe9gsvc
  Running command git clone -q git://github.com/jaidedai/easyocr.git /tmp/pip-req-build-nqe9gsvc
     |████████████████████████████████| 47.6 MB 3.9 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.5.62
    Uninstalling opencv-python-headless-4.5.5.62:
      Successfully uninstalled opencv-python-headless-4.5.5.62


In [9]:
#!pip uninstall opencv-python-headless 
!pip install opencv-python-headless==4.1.2.30

     |████████████████████████████████| 21.8 MB 1.5 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.5.4.60
    Uninstalling opencv-python-headless-4.5.4.60:
      Successfully uninstalled opencv-python-headless-4.5.4.60


In [10]:
import easyocr

##EASY OCR MODEL


In [11]:
reader = easyocr.Reader(['vi']) # this needs to run only once to load the model into memory
#result = reader.readtext('test.jpg')

In [12]:
import cv2
from google.colab.patches import cv2_imshow

In [13]:
from google.colab import files

In [14]:
def SingleImageLabel(result, file_name):
  label = []
  for i in range(len(result)):
    temp_dct = {}
    temp_dct.update({"transcription":result[i][1]})
    temp_dct.update({"points":result[i][0]})
    temp_dct.update({"difficult":False})
    label.append(temp_dct)
  return label

def UpdateLabel(label, filename):
  label_file = label_file + model + str("/")+str(filename)+str("\t")+str(label)+str("\n")
  return label_file

def SaveLabel(path, label_file):
  #print(label_file)
  f = open(path, "w")
  f.write(label_file)
  f.close()

#get all file names
import os
#model = "EASYOCR"
#path = "/content/drive/MyDrive/AT SCHOOL/HK7/CS114/Final Project/Prediction/Test/"+model+"/"
lst_file =[]
#for filename in os.listdir(path):
 # lst_file.append(filename)

def save_segmentation_image(path, lst_file):
  leng = len(lst_file)
  for i in range(leng):
    print(i)
    image = cv2.imread(path + lst_file[i])
    try:
      after = segment_single_images(image, False)
      cv2.imwrite(path+lst_file[i].split(".")[0]+ "non_backgr.jpg", after)
      print(i)
    except: 
      print("error>>",i)
      
def handle_bb(path, model, lst_file, label_file):
  for i in range(len(lst_file)):
    result = reader.readtext(path + lst_file[i])

    label = SingleImageLabel(result, lst_file[i])
    #print("label  ", label)

    #updatelabel
    print("UpdateLabel")
    label_file = label_file + model + str("/")+str(lst_file[i])+str("\t")+str(label)+str("\n")

  SaveLabel(path+"Label.txt", label_file)
  print("Done")

In [15]:
#label_file = ""
#handle_bb(path, "EASYOCR", lst_file, label_file)

In [16]:
#replace to putText
'''
from PIL import Image, ImageDraw, ImageFont
from typing import Tuple
def cv2_img_add_text(img, text, left_corner: Tuple[int, int],
                     text_rgb_color=(255, 0, 0), text_size=24, font='arial.ttc', **option):
    pil_img = img
    if isinstance(pil_img, np.ndarray):
        pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    draw = ImageDraw.Draw(pil_img)
    font_text = ImageFont.truetype(font=font, size=text_size, encoding=option.get('encoding', 'utf-8'))
    draw.text(left_corner, text, text_rgb_color, font=font_text)
    cv2_img = cv2.cvtColor(np.asarray(pil_img), cv2.COLOR_RGB2BGR)
    if option.get('replace'):
        img[:] = cv2_img[:]
        return None
    return cv2_img'''

"\nfrom PIL import Image, ImageDraw, ImageFont\nfrom typing import Tuple\ndef cv2_img_add_text(img, text, left_corner: Tuple[int, int],\n                     text_rgb_color=(255, 0, 0), text_size=24, font='arial.ttc', **option):\n    pil_img = img\n    if isinstance(pil_img, np.ndarray):\n        pil_img = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))\n    draw = ImageDraw.Draw(pil_img)\n    font_text = ImageFont.truetype(font=font, size=text_size, encoding=option.get('encoding', 'utf-8'))\n    draw.text(left_corner, text, text_rgb_color, font=font_text)\n    cv2_img = cv2.cvtColor(np.asarray(pil_img), cv2.COLOR_RGB2BGR)\n    if option.get('replace'):\n        img[:] = cv2_img[:]\n        return None\n    return cv2_img"

In [17]:
# coding=utf-8 show detect & recognize text
'''import cv2
import numpy as np
import matplotlib.pyplot as plt
image = cv2.imread('/content/photo_13@13-01-2022_22-50-55.jpg')
reader = easyocr.Reader(['vi'])
res = reader.readtext(image) 
for (bbox, text, prob) in res: 
  # unpack the bounding box
  (tl, tr, br, bl) = bbox
  tl = (int(tl[0]), int(tl[1]))
  tr = (int(tr[0]), int(tr[1]))
  br = (int(br[0]), int(br[1]))
  bl = (int(bl[0]), int(bl[1]))
  cv2.rectangle(image, tl, br, (0, 255, 0), 2)
  #IMG_SHAPE=(200, 520, 3)
  #np_img = np.ones(IMG_SHAPE, dtype=np.uint8) * 255
  #np_img = cv2_img_add_text(np_img, text, (tl[0],tl[1] - 10), text_rgb_color=(255, 0, 0), text_size=2)
  cv2.putText(image, text, (tl[0], tl[1] - 10),
    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)
plt.rcParams['figure.figsize'] = (16,16)
plt.imshow(image)'''

"import cv2\nimport numpy as np\nimport matplotlib.pyplot as plt\nimage = cv2.imread('/content/photo_13@13-01-2022_22-50-55.jpg')\nreader = easyocr.Reader(['vi'])\nres = reader.readtext(image) \nfor (bbox, text, prob) in res: \n  # unpack the bounding box\n  (tl, tr, br, bl) = bbox\n  tl = (int(tl[0]), int(tl[1]))\n  tr = (int(tr[0]), int(tr[1]))\n  br = (int(br[0]), int(br[1]))\n  bl = (int(bl[0]), int(bl[1]))\n  cv2.rectangle(image, tl, br, (0, 255, 0), 2)\n  #IMG_SHAPE=(200, 520, 3)\n  #np_img = np.ones(IMG_SHAPE, dtype=np.uint8) * 255\n  #np_img = cv2_img_add_text(np_img, text, (tl[0],tl[1] - 10), text_rgb_color=(255, 0, 0), text_size=2)\n  cv2.putText(image, text, (tl[0], tl[1] - 10),\n    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 0, 0), 2)\nplt.rcParams['figure.figsize'] = (16,16)\nplt.imshow(image)"

##PREDICT

In [18]:
%cd $easyocr_pre_dir

/content/drive/.shortcut-targets-by-id/1N-Kteu_6UzW5J7a9vCpt2u6ww5uhGJfh/AT SCHOOL/HK7/CS114/Final Project/Prediction/EASYOCR/points_text


In [19]:
lst_file =[]
for filename in os.listdir(test_dir):
  lst_file.append(filename)
try:
  lst_file.remove("Cache.cach")
  lst_file.remove("Label.txt")
  lst_file.remove("fileState.txt")
  lst_file.remove(".ipynb_checkpoints")
except: 
  pass

In [20]:
len(lst_file)

90

In [21]:
lst_file.pop(-1)

'photo_353@05-01-2022_08-45-59.jpg'

In [22]:
def predict(data_path,predict_path, lst_file):
  label_file = ""
  reader = easyocr.Reader(['vi'])
  #for i in range(len(lst_file)):
  for i in tqdm (range(len(lst_file)), desc="Loading…", ascii=False, ncols=100):
    try:
      result = reader.readtext(data_path + lst_file[i])
      label = SingleImageLabel(result, lst_file[i])
    #print("label  ", label)
    #updatelabel
    #print("UpdateLabel ",i,">>>>",lst_file[i])
      label_file = label_file + "EASYOCR" + str("/")+str(lst_file[i])+str("\t")+str(label)+str("\n")
    except:
      pass

  SaveLabel(predict_path+"Label.txt", label_file)
  print("Done")
  return label_file

In [23]:
label_file = predict(test_dir,easyocr_pre_dir, lst_file)

Loading…: 100%|█████████████████████████████████████████████████████| 89/89 [02:56<00:00,  1.99s/it]


Done


##EVALUATE

In [24]:
#predict_label_path = "/content/drive/MyDrive/AT SCHOOL/HK7/CS114/Final Project/Prediction/EASYOCR/Label.txt"
groundtruth_label_path = "/content/drive/MyDrive/AT SCHOOL/HK7/CS114/Final Project/Data/Train_test_val/Label/Test_Label.txt"

In [25]:
test_dir

'/content/drive/MyDrive/AT SCHOOL/HK7/CS114/Final Project/Data/Train_test_val/Test/'

In [26]:
threshold = 0.5

In [27]:
def convertDfTrue(path):
  f = open(path, "r")
  lst = []
  for x in f:
    temp = x.split("\t")
    lst.append(temp)

  df = pd.DataFrame(lst) 
  df.columns=["name", "detect"]
  df["points"] = np.nan
  df["text"]= np.nan
  
  final_lst =[]
  for i in tqdm (range(len(df)), desc="Loading…", ascii=False, ncols=75):
    points = []
    text= []
    temp = df["detect"].iloc[i].split('"difficult": false}')
    temp.pop(-1)
    for j in range(len(temp)):
      temp1 = temp[j].strip("[, ")+"}"
      try:
        dct_temp = ast.literal_eval(temp1)
        points.append(dct_temp["points"])
        text.append(dct_temp["transcription"])
      except: 
        temp2 = temp1.split(', "difficult": true}, ')
        temp2[0]=temp2[0]+"}"
        for h in temp2:
          dct_temp = ast.literal_eval(h)
          points.append(dct_temp["points"])
          text.append(dct_temp["transcription"])
    
    final_lst.append([[df["name"].iloc[i]],points ,text ])
  _df = pd.DataFrame(final_lst)
  #_df = _df.set_index(_df[0])
  #_df = _df.drop([0], axis=1)
  _df.columns = ["name","points", "text"]
  for i in range(len(_df)):
    _df["name"].iloc[i] = _df["name"].iloc[i][0]
    _df["name"].iloc[i] = _df["name"].iloc[i].replace("Test/","")
  #print(_df)
  return _df

In [28]:
def convertDfPre(str_):
  lst = []
  str_ = str_.split("\n")
  str_.pop(-1)
  for i in str_: 
    temp = i.split("\t")
    lst.append(temp)

  df = pd.DataFrame(lst) 
  df.columns=["name", "detect"]
  df["points"] = np.nan
  df["text"]= np.nan
  final_lst =[]
  for i in tqdm (range(len(df)), desc="Loading…", ascii=False, ncols=75):
    points = []
    text= []
    temp = df["detect"].iloc[i].split(", 'difficult': False},")
    #temp.pop(-1)
    for j in range(len(temp)):
      temp1 = temp[j].strip("[, ")+"}"
      try:
        dct_temp = ast.literal_eval(temp1)
        points.append(dct_temp["points"])
        text.append(dct_temp["transcription"])
      except: 
        temp2 = temp1.replace(", 'difficult': False}]}", "")
        temp2 = temp2+"}"
        dct_temp = ast.literal_eval(temp2)
        points.append(dct_temp["points"])
        text.append(dct_temp["transcription"])
    
    final_lst.append([[df["name"].iloc[i]],points ,text ])
  _df = pd.DataFrame(final_lst)
  print(_df.columns)
  #_df = _df.set_index(_df[0])
  #_df = _df.drop([0], axis=1)
  _df.columns = ["name","points", "text"]
  
  for i in range(len(_df)):
    _df["name"].iloc[i] = _df["name"].iloc[i][0]
    _df["name"].iloc[i] = _df["name"].iloc[i].replace("EASYOCR/","")
  return _df

In [29]:
print("predicted list")
final_pre_df = convertDfPre(label_file)
print("groundtrurh list")
final_true_df = convertDfTrue(groundtruth_label_path)

predicted list


Loading…: 100%|███████████████████████████| 89/89 [00:00<00:00, 992.70it/s]

RangeIndex(start=0, stop=3, step=1)
groundtrurh list



Loading…: 100%|██████████████████████████| 90/90 [00:00<00:00, 2045.17it/s]


In [30]:
def CalculateIou(lst1, lst2):
  polygon1 = Polygon(np.array(lst1))
  polygon2 = Polygon(np.array(lst2))
  intersect = polygon1.intersection(polygon2).area
  union = polygon1.union(polygon2).area
  iou = intersect / union
  #print(iou)
  return iou

In [31]:
def CalculateScore_image(name,threshold ): 
  true = final_true_df[final_true_df["name"] == name]["points"].values
  pre  = final_pre_df[final_pre_df["name"] == name]["points"].values

  temp_lst = []
  for i in range(len(pre[0])):
    for j in range(len(true[0])):
      iou = CalculateIou(pre[0][i], true[0][j])
      temp_lst.append([i, j, iou])
  temp_df = pd.DataFrame(temp_lst, columns = ["pol_pre", "pol_true", "iou"])
  temp_df = temp_df.sort_values(["iou", "pol_true"], ascending=False)
  temp_df = temp_df.reset_index()

  overlap = {}
  temp_df["select"]=np.nan
  temp_df["evaluate"]  = np.nan
  for i in range(len(pre[0]) *len(true[0])):
    if temp_df["pol_true"].iloc[i] not in overlap.keys():
      overlap.update({temp_df["pol_true"].iloc[i]:temp_df["pol_pre"].iloc[i]})
      temp_df["select"].iloc[i] =1
    
  for i in range(len(pre[0]) *len(true[0])):
    if (temp_df["iou"].iloc[i] >= threshold and temp_df["select"].iloc[i] == 1.0):
      temp_df["evaluate"].iloc[i] = "True Positive"
    if (temp_df["iou"].iloc[i] >= threshold and temp_df["select"].iloc[i] != 1.0):
      temp_df["evaluate"].iloc[i] = "False Positive"
    if (temp_df["iou"].iloc[i] <= threshold and temp_df["select"].iloc[i] == 1.0):
      temp_df["evaluate"].iloc[i] = "False Negative"
  #temp_df = temp_df[temp_df["select"] == 1]

  #iou = list(temp_df["iou"])
  #REDUNDANT
  redundant = []
  temp_re = set(list(temp_df["pol_pre"]))
  for i in temp_re: 
    if i not in list(temp_df[temp_df["select"]==1]["pol_pre"]):
      #iou.append(0)
      redundant.append("FP")

  #MISS
  miss = []
  temp_miss = set(list(temp_df["pol_true"]))
  for i in temp_miss: 
    if i not in list(temp_df[temp_df["select"]==1]["pol_true"]):
      #iou.append(0)
      miss.append("FN")

  FN = len(miss) + len(temp_df[temp_df["evaluate"]=="False Negative"])
  TP = len(temp_df[temp_df["evaluate"]=="True Positive"]) 
  FP = len(temp_df[temp_df["evaluate"]=="False Positive"]) +len(redundant)
  precision = TP/(TP + FP)
  recall    = TP/(TP + FN)
  final_df = temp_df[temp_df["evaluate"]== "True Positive"]
  final_df.to_csv(easyocr_pre_dir+name+".csv", index = None)
  return precision, recall

In [32]:
def CalculateIou_Test(lst_file, thresold):
  precision = []
  recall = []
  #lst_file.pop(-1)
  for i in tqdm (range(len(lst_file)), desc="Loading…", ascii=False, ncols=75): 
  #for i in range(len(lst_file)):
    #print(i)
    try:
      p, r = CalculateScore_image(lst_file[i], threshold)
      precision.append(p)
      recall.append(r)
    except: 
      pass
  pre = mean(precision)
  re  = mean(recall)
  return pre, re

In [33]:
#lst_file.pop(-1)

In [34]:
final_pre_df.iloc[0]

name                       photo_43@05-01-2022_08-41-24.jpg
points    [[[386, 274], [532, 274], [532, 300], [386, 30...
text      [BỌ GIÁO DUC VÀ, Cuumg   tUPT_rg' PInh hhiér, ...
Name: 0, dtype: object

In [35]:
final_true_df.iloc[0]

name                        photo_1@05-01-2022_08-40-54.jpg
points    [[[487, 176], [749, 212], [746, 234], [484, 19...
text      [BỘ GIÁO DỤC VÀ ĐÀO TẠO, CÔNG NGHỆ, NÔNG, LÂM,...
Name: 0, dtype: object

In [36]:
#CalculateScore_image()

In [37]:
precision50, recall50 = CalculateIou_Test(lst_file, threshold)

Loading…:   0%|                                     | 0/89 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
Loading…: 100%|████████████████████████████| 89/89 [01:09<00:00,  1.28it/s]


In [38]:
print("Precision: ",precision50,"\n", "Recall: ", recall50)

Precision:  0.3786230433778046 
 Recall:  0.7140229620566699


In [39]:
(2*precision50*recall50)/(precision50+recall50)

0.49484562354306544

##save list_label

In [40]:
#label_file = "\n".join(label_file)

In [41]:
f = open(easyocr_pre_dir+"label_file.txt", "w")
f.write(label_file)
f.close()

##Text recognition score

In [42]:
final_pre_df = convertDfPre(label_file)

Loading…: 100%|███████████████████████████| 89/89 [00:00<00:00, 951.39it/s]

RangeIndex(start=0, stop=3, step=1)


In [43]:
final_true_df = convertDfTrue(groundtruth_label_path)

Loading…: 100%|██████████████████████████| 90/90 [00:00<00:00, 2208.20it/s]


In [44]:
point_text = "/content/drive/MyDrive/AT SCHOOL/HK7/CS114/Final Project/Prediction/EASYOCR/points_text/"

In [45]:
def Calculate_score_text(final_true_df, final_pre_df):
  #cer_lst = []
  #wer_lst = []
  true = []
  pre = []
  for i in range(len(lst_file)):
    name = lst_file[i]
    df = pd.read_csv(point_text+name+".csv")
    true_lst = []
    pre_lst  = []
    for i in range(len(df)):
      true_lst.append(final_true_df[final_true_df["name"]==name]["text"].values[0][df["pol_true"].iloc[i]])
      pre_lst.append(final_pre_df[final_pre_df["name"]==name]["text"].values[0][df["pol_pre"].iloc[i]])
    true.append(true_lst)
    pre.append(pre_lst)
    #w = wer(true_lst, pre_lst)
    #c = cer(true_lst, pre_lst)
    #cer_lst.append(c)
    #wer_lst.append(w)
  #_cer = mean(cer_lst)
  #_wer = mean(wer_lst)
  return true, pre


In [46]:
true = []
pre = []
true,pre =  Calculate_score_text(final_true_df, final_pre_df)

In [47]:
import numpy as np
from sklearn.metrics import precision_recall_fscore_support
y_true = np.array(true[0])
y_pred = np.array(pre[0])

In [48]:
true[2][2].split(" ")

['CÓ', 'ĐÁP', 'ÁN']

In [49]:
pre[2][2].split(" ")

['56ĐÁPÁN']

In [50]:
def get_lst_true_pre_each_image(true, pre):
  lst_true = []
  lst_pre = []
  for i in range(len(true)):
    #print(i)
    temp_true = true[i].split(" ")
    temp_pre = pre[i].split(" ")
    
    for j in range(len(temp_true)):
      lst_true.append(temp_true[j])
    for j in range(len(temp_pre)):
      lst_pre.append(temp_pre[j])
    hon = len(lst_true) - len(lst_pre)
    if hon > 0: 
      for j in range(hon):
        lst_pre.append("")
    else:
      for j in range(hon*(-1)):
        lst_true.append("")
  return lst_true, lst_pre

In [51]:
get_lst_true_pre_each_image(true[1], pre[1])

(['MẸ', 'KỂ', 'CON', 'NGHE', 'TRUYÊNK', '101', 'VÂN', 'ANH'],
 ['MẸ', 'KỂ', 'CON', 'NGHE', 'TRỦYỆN', '70', 'VêN', 'ANH'])

In [52]:
def cal_test_score(true, pre):
  lst_score = []
  for i in range(len(true)):
    lst_true, lst_pre = get_lst_true_pre_each_image(true[i], pre[i])
    score = list(precision_recall_fscore_support(lst_true, lst_pre, average='micro'))
    #print(i,">>>",score)
    lst_score.append(score[2])
  f1 = mean(lst_score)
  return f1

In [53]:
h_mean  = cal_test_score(true, pre)
print("H-mean score ", h_mean)

H-mean score  0.4706411998377093
